In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://www.smf.org.mx/rmf/'
url_root = 'https://www.smf.org.mx/rmf/'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', id='Capa_Vol_31_1_2013')
volumen=box.findAll('div', id='issue')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

AttributeError: 'NoneType' object has no attribute 'findAll'

In [ ]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='Capa_Vol_31_1_2013')
    volumen=box.findAll('div', id='issue')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Fitopatología'
    a='VI. Biotecnología y Ciencias Agropecuarias'
    tem='Etiología, Manejo Fitosanitario, Epidemiología, Biología, Taxonomía, Control de Hongos, Bacterias, Virus, Nematodos y Otros Organismos Fitopatógenos'
    url='http://rmf.smf.org.mx/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('td', class_='tocTitle').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:4]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revista97.csv', index=False)